In [ ]:
import logging
import pandas as pd
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')

    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def get_sectors(conn, cur, state):
    query = f"SELECT cod_ibge FROM public.setores_censitarios WHERE CAST(cod_ibge AS TEXT) LIKE '{state}%';"
    
    try:
        cur.execute(query)
        result = [row[0] for row in cur.fetchall()]
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def get_data(conn, cur, sector):
    query = f" \
        SELECT \
            PRM.\"00_04\"::NUMERIC / NULLIF(PRF.\"00_04\"::NUMERIC, 0) AS RS_00_04, \
            PRM.\"05_09\"::NUMERIC / NULLIF(PRF.\"05_09\"::NUMERIC, 0) AS RS_05_09, \
            PRM.\"10_14\"::NUMERIC / NULLIF(PRF.\"10_14\"::NUMERIC, 0) AS RS_10_14, \
            PRM.\"15_19\"::NUMERIC / NULLIF(PRF.\"15_19\"::NUMERIC, 0) AS RS_15_19, \
            PRM.\"20_24\"::NUMERIC / NULLIF(PRF.\"20_24\"::NUMERIC, 0) AS RS_20_24, \
            PRM.\"25_29\"::NUMERIC / NULLIF(PRF.\"25_29\"::NUMERIC, 0) AS RS_25_29, \
            PRM.\"30_34\"::NUMERIC / NULLIF(PRF.\"30_34\"::NUMERIC, 0) AS RS_30_34, \
            PRM.\"35_39\"::NUMERIC / NULLIF(PRF.\"35_39\"::NUMERIC, 0) AS RS_35_39, \
            PRM.\"40_44\"::NUMERIC / NULLIF(PRF.\"40_44\"::NUMERIC, 0) AS RS_40_44, \
            PRM.\"45_49\"::NUMERIC / NULLIF(PRF.\"45_49\"::NUMERIC, 0) AS RS_45_49, \
            PRM.\"50_54\"::NUMERIC / NULLIF(PRF.\"50_54\"::NUMERIC, 0) AS RS_50_54, \
            PRM.\"55_59\"::NUMERIC / NULLIF(PRF.\"55_59\"::NUMERIC, 0) AS RS_55_59, \
            PRM.\"60_64\"::NUMERIC / NULLIF(PRF.\"60_64\"::NUMERIC, 0) AS RS_60_64, \
            PRM.\"65_69\"::NUMERIC / NULLIF(PRF.\"65_69\"::NUMERIC, 0) AS RS_65_69, \
            PRM.\"70_74\"::NUMERIC / NULLIF(PRF.\"70_74\"::NUMERIC, 0) AS RS_70_74, \
            PRM.\"75_79\"::NUMERIC / NULLIF(PRF.\"75_79\"::NUMERIC, 0) AS RS_75_79, \
            PRM.\"80_84\"::NUMERIC / NULLIF(PRF.\"80_84\"::NUMERIC, 0) AS RS_80_84, \
            PRM.\"85_89\"::NUMERIC / NULLIF(PRF.\"85_89\"::NUMERIC, 0) AS RS_85_89, \
            PRM.\"90_94\"::NUMERIC / NULLIF(PRF.\"90_94\"::NUMERIC, 0) AS RS_90_94, \
            PRM.\"95_99\"::NUMERIC / NULLIF(PRF.\"95_99\"::NUMERIC, 0) AS RS_95_99, \
            PRM.\"100\"::NUMERIC / NULLIF(PRF.\"100\"::NUMERIC, 0) AS RS_100 \
        FROM PUBLIC.populacao_residente_mas AS PRM \
        INNER JOIN PUBLIC.populacao_residente_fem AS PRF ON  \
            PRM.id_setor_censitario = PRF.id_setor_censitario \
        WHERE PRM.id_setor_censitario = {sector};"

    try:
        cur.execute(query)
        result = cur.fetchall()
    except psycopg2.Error as e:
        logger.error(f"Failed: {e}")
        
    return result

In [ ]:
def insert_data(conn, cur, sector, data):
    values = [data[0]] + ['NULL' if item is None else round(float(item), 2) for item in data[1:][0]]
    values = ', '.join(map(str, [item for item in values]))

    query = f" \
        INSERT INTO public.razao_sexo( \
            id_setor_censitario, \
            \"00_04\", \"05_09\", \"10_14\", \"15_19\", \"20_24\", \"25_29\", \"30_34\", \
            \"35_39\", \"40_44\", \"45_49\", \"50_54\", \"55_59\", \"60_64\", \"65_69\", \
            \"70_74\", \"75_79\", \"80_84\", \"85_89\", \"90_94\", \"95_99\", \"100\") \
        VALUES ({values});"
    
    try:
        cur.execute(query)
    except psycopg2.IntegrityError as e:
        conn.rollback()
        logger.error(f"Failed: {e}")
    else:
        conn.commit()

---

In [ ]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [ ]:
conn, cur = open_connection()

In [ ]:
sectors = get_sectors(conn, cur, 12) # AC

In [ ]:
logger.info(f"{len(sectors)} found.")

In [ ]:
%%time
for sector in sectors:
    data = get_data(conn, cur, sector)
    
    if len(data) > 0:
        data.insert(0, sector)
        insert_data(conn, cur, sector, data)

In [ ]:
close_connection(conn, cur)